# IMOS Ocean Radar Data Query

**Acknowlegement** &mdash; This  is based on the Ipython material from <a href:'https://github.com/DamienIrving/CV/blob/master/CV.md'>Damien Irving</a>.

## Installing modules

A module is a Python object with arbitrarily named attributes that you can bind and reference. A module is a fit-2-purpose piece of code which defines 

+ functions, 
+ classes, 
+ variables, 
+ a runnable code.

Installing a module is usually done using Python Package Installer: <code>pip</code>. 

All the modules that you will require for this UoS have already been installed on the server, so you will have nothing to install but you will still have to load the modules on the IPython environment.

We will need **netCDF** libray (<a ref='http://www.unidata.ucar.edu/netcdf/'>Network Common Data Form</a>) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data. The project homepage is hosted by the **Unidata** program at the University Corporation for Atmospheric Research (**UCAR**).

Loading a module is straight forward:

In [ ]:
%matplotlib inline
from netCDF4 import Dataset
from netCDF4 import num2date
from IPython.core.display import HTML
import matplotlib.pyplot as plt

# Data server = Data access ;-)

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('7yTpv70gkGE')

## In a nutshell...

* Data access is provided via **HTTP** and **OPeNDAP** by **THREDDS** Data Server and **FTP**. 
* Most data providers are using interactive visualization of data sets with zoomable maps. 

To understand data server you need to be familiar with these key transfer protocols:

#### HTTP: Hypertext Transfer Protocol
#### OPenDAP: Open-source Project for a Network Data Access Protocol
#### THREDDS: Thematic Real-Time Environmental Distributed Data Services
#### FTP: File Transfer Protocol
#### LAS: Live Access Server

# Get the data from IMOS

In [ ]:
# Turquoise bay
acorn_URL = 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/eMII/demos/ACORN/monthly_gridded_1h-avg-current-map_non-QC/TURQ/2012/IMOS_ACORN_V_20121001T000000Z_TURQ_FV00_monthly-1-hour-avg_END-20121029T180000Z_C-20121030T160000Z.nc.gz'
# Other ACORN data as an example:
# South Australia Gulf
#acorn_URL = 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/ACORN/monthly_gridded_1h-avg-current-map_non-QC/SAG/2015/IMOS_ACORN_V_20150101T003000Z_SAG_FV00_monthly-1-hour-avg_END-20150131T233000Z_C-20150201T235953Z.nc'
# Bonney Coast
#acorn_URL = 'http://thredds.aodn.org.au/thredds/dodsC/IMOS/ACORN/monthly_gridded_1h-avg-current-map_non-QC/BONC/2015/IMOS_ACORN_V_20150101T000000Z_BONC_FV00_monthly-1-hour-avg_END-20150131T190000Z_C-20150202T000206Z.nc'

acorn_DATA = Dataset(acorn_URL)

The first thing to notice is the distinctive **Data Reference Syntax** (DRS) associated with the file. The staff at IMOS have archived the data according to the following directory structure:

**http://thredds.aodn.org.au/thredds/dodsC/project/organisation/collection/facility/data-type/site-code/year/**

From this we can deduce, without even inspecting the contents of the file, that we have data from the IMOS project that is run by the eMarine Information Infrastructure (eMII). It was collected in 2012 at the Turquoise Coast, Western Australia (TURQ) site of the Australian Coastal Ocean Radar Network (ACORN), which is a network of high frequency radars that measure the ocean surface current (see this <a href='http://researchdata.ands.org.au/imos-acorn-turquoise-australia-australia/442676'>page</a> on the Research Data Australia website for a nice overview of the dataset).

<br/>
<small>While it's unlikely that your research will ever involve cataloging data from such a large observational network, it's still a very good idea to develop your own personal DRS for the data you do have. This often involves investing some time at the beginning of a project to think carefully about the design of your directory and file name structures, as these can be very hard to change later on (a good example is the <a ref='http://cmip-pcmdi.llnl.gov/cmip5/docs/cmip5_data_reference_syntax.pdf'>DRS</a> used by the Climate Model Intercomparison Project). The combination of bash shell wildcards and a well planned DRS is one of the easiest ways to make your research more efficient and reliable.
</small>



The data type has a sub-DRS of its own, which tells us that the data represents the 1-hourly average surface current for a single month (October 2012), and that it is archived on a regularly spaced spatial grid and has not been quality controlled. The file is located in the "demos" directory, as it has been generated for the purpose of providing an example for users in the very helpful <a ref='http://portal.aodn.org.au/aodn/'>Australian Ocean Data Network</a> (AODN) <a href='https://github.com/aodn/imos-user-code-library'> user code library</a>.

Just in case the file gets separated from this informative directory stucture, much of the information is repeated in the file name itself, along with some more detailed information about the start and end time of the data, and the last time the file was modified.

**project_facility_V_time-start_site-code_FV00_data-type_time-end_modified.nc.gz**

In the first instance this level of detail seems like a bit of overkill, but consider the scope of the IMOS data archive. It is the final resting place for data collected by the entire national array of oceanographic observing equipment in Australia, which monitors the open oceans and coastal marine environment covering physical, chemical and biological variables. Since the data are so well labelled, locating all monthly timescale ACORN data from the Turquoise Coast and Rottnest Shelf sites (which represents hundreds of files) would be as simple as typing the following at the command line:

**ls */ACORN/monthly_*/{TURQ,ROT}/\*/\*.nc**

## Check the loaded ACORN dataset

In [ ]:
print acorn_DATA

# Query the dataset

The great thing about netCDF files is that they contain <a href=http://en.wikipedia.org/wiki/Metadata>metadata</a> - that is, data about the data. There are global attributes that give information about the file as a whole, while each variable also has its own attributes.

The file contains the following variables:

In [ ]:
print acorn_DATA.variables.keys()

The 'u' means each variable name is represented by a Unicode string.

In [ ]:
print 'These are the attributes of the time axis:'
print acorn_DATA.variables['TIME']
print 'These are some of the time values:'
print acorn_DATA.variables['TIME'][0:10]

The raw time values are fairly meaningless, but we can use the time attributes to convert them to a more meaningful format...

In [ ]:
units = acorn_DATA.variables['TIME'].units
calendar = acorn_DATA.variables['TIME'].calendar

times = num2date(acorn_DATA.variables['TIME'][:], units=units, calendar=calendar)
print times[0:10]

<blockquote>
    <h2>Climate and Forecast (CF) metadata convention</h2>
    <p>When performing simple data analysis tasks on netCDF files, command line tools like the Climate Data Operators (<a href="https://code.zmaw.de/projects/cdo">CDO</a>) are often a better alternative to writing your own functions in Python. However, let's put ourselves in the shoes of the developers of CDO for a minute. In order to calculate the time mean of a dataset for a given start and end date (for example), CDO must first identify the units of the time axis. This isn't as easy as you'd think, since the creator of the netCDF file could easily have called the units attribute measure, or scale, or something else completely unpredictable. They could also have defined the units as weeks since 1-01-01 00:00:00 or milliseconds after 1979-12-31. Obviously what is needed is a standard method for defining netCDF attributes, and that’s where the <a href='http://cf-pcmdi.llnl.gov/'>Climate and Forecast (CF) metadata convention</a> comes in.</p>
    <p>The CF metadata standard was first defined back in the early 2000s and has now been adopted by all the major institutions and projects in the weather/climate sciences. There's a nice <a href='http://drclimate.wordpress.com/2014/06/09/are-you-cf-compliant/'>blog post</a> on the topic if you'd like more information, but for the most part you just need to be aware that if a tool like CDO isn't working, it might be because your netCDF file isn't CF compliant.</p>
</blockquote>

# Calculating the current speed

For the sake of example, let's say that our data file contained the zonal (east/west; 'UCUR') and meridional (north/south; 'VCUR') surface current components, but not the total current speed. To calculate it, we first need to assign a variable to the zonal and meridional current data.

In [ ]:
uData = acorn_DATA.variables['UCUR'][:,:,:]
vData = acorn_DATA.variables['VCUR'][:,:,:]

Both <i>uData</i> and <i>vData</i> are a special type of numpy array known as a masked array, whereby some of the points in the time/latitude/longitude grid have missing (or masked) values. Just as with a normal numpy array, we can check the shape of our data (in fact, masked arrays can do everything normal numpy arrays can do and more).

In [ ]:
print type(uData)
print uData.shape

In other words, 493 time steps, 55 latitudes and 57 longitudes. We can now go ahead and calculate the current speed.

In [ ]:
spData = (uData**2 + vData**2)**0.5

# Viewing Turquoise Bay uploaded dataset

It's a good idea to regularly view your data throughout the code development process, just to ensure nothing crazy has happened along the way. Below is a code except from this <a href='https://github.com/aodn/imos-user-code-library/blob/master/Python/demos/acorn.py'>example</a> in the AODN user code library, which simply plots one of the 493 timesteps.

In [ ]:
LAT = acorn_DATA.variables['LATITUDE']
LON = acorn_DATA.variables['LONGITUDE']
TIME = acorn_DATA.variables['TIME']

# Only one time value is being plotted. modify timeIndex if desired (value between 0 and length(timeData)-1 )
timeIndex = 4
speedData = spData[timeIndex,:,:]
latData = LAT[:]
lonData = LON[:]

# sea water U and V components
uData = acorn_DATA.variables['UCUR'][timeIndex,:,:]
vData = acorn_DATA.variables['VCUR'][timeIndex,:,:]
units = acorn_DATA.variables['UCUR'].units

figure1 = plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.pcolor(lonData , latData, speedData)
cbar = plt.colorbar()
cbar.ax.set_ylabel('Current speed in ' + units)

plt.title(acorn_DATA.title + '\n' + num2date(TIME[timeIndex], TIME.units, TIME.calendar).strftime('%d/%m/%Y'))
plt.xlabel(LON.long_name + ' in ' + LON.units)
plt.ylabel(LAT.long_name + ' in ' + LAT.units)

#plot velocity field
Q = plt.quiver(lonData[:], latData[:], uData, vData, units='width')
plt.show()